In [3]:
import numpy as np
import cv2

# Quantidade de pontos no tabuleiro
nx, ny = 5, 8
objpoints = []

def getInfo(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
    duration = frame_count / fps
    return int(fps), int(frame_count), int(duration)

def concatenar_saida(images): 
    return cv2.vconcat(
        [cv2.hconcat(image) for image in images]
    )

def binarizar(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blur = cv2.medianBlur(frame, 9)
    ret, binarizada = cv2.threshold(blur, 4, 255, cv2.THRESH_BINARY)
    binarizada = cv2.dilate(binarizada, cv2.getStructuringElement(
        cv2.MORPH_ELLIPSE, (3,3)
    ), iterations = 2)
    binarizada = cv2.resize(binarizada, (320, 320))
    return binarizada

def getCorners(frame, nx, ny):
    ret, corners = cv2.findChessboardCorners(frame, (nx, ny))
    # print(corners)
    return ret, corners

# def getPlanoDeFundo(frame):
#     fgmask = cv2.createBackgroundSubtractorMOG2().apply(frame)
#     fgmask = cv2.cvtColor(fgmask, cv2.COLOR_GRAY2BGR)
#     cv2.imshow('fundo', fgmask)
#     return fgmask

def desenharPontos(frame_inicial, ret, corners, nx, ny, objpoints):
    objp = np.zeros((nx*ny,2), np.float32)
    objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)
    
    if ret:
        pontos = np.int0(corners)
        for c in pontos:
            x,y = c.ravel()
            cv2.circle(frame_inicial, (x,y), 3, (0,0,255), -1)
        objpoints.append(objp)
        # cv2.imshow('teste',frame_inicial)

def getFrame(video, posicao):
    video.set(cv2.CAP_PROP_POS_FRAMES, posicao)
    res, frame = video.read()
    # x, y = frame.shape[:2]
    frame = cv2.resize(frame, (320, 320))
    return res, frame

video = cv2.VideoCapture('xadrez.mp4')
fps, frame_count, duration = getInfo(video)

# frame com o tabuleiro
_, frame_inicial = getFrame(video, 80)
# frame com o fundo
_, frame_fundo = getFrame(video, 150)
# Seta para o momento da bolinha (não precisamos percorrer o vídeo inteiro)
# Converte para grayscale
gray = cv2.cvtColor(frame_inicial, cv2.COLOR_BGR2GRAY)
# Peg os cantos (nx, ny)
ret, corners = getCorners(gray, nx, ny)
# Desenha os pontos no tabuleiro (opcional)
desenharPontos(frame_inicial, ret, corners, nx, ny, objpoints)

# print(np.float32(objpoints), corners)

objpoints = np.float32(objpoints)
# Calcular aqui a homografia
h, mask = cv2.findHomography(corners, objpoints, cv2.RANSAC)
# print(h)

# video.set(cv2.CAP_PROP_POS_FRAMES, 220)
# Loop no vídeo (frame a frame)
while video.isOpened():
    ok, frame = video.read()
    if frame is None:
        break
    frame = cv2.resize(frame, (320,320))
    
    bolinha = cv2.subtract(frame, frame_fundo)
    binarizada = binarizar(bolinha)
    binarizada = cv2.cvtColor(binarizada, cv2.COLOR_GRAY2BGR)

    # encontrar o menor ponto Y (branco)
    # desenhar no frame bolinha
    # smallest = binarizada.min(axis=(0, 1))
    # largest = binarizada.max(axis=(0, 1))

    white = [255,255,255]
    # X = np.where(np.all(binarizada==white,axis=1))
    # if len(X) != 0:
    #     cv2.circle(binarizada, (y,y), 3, (0,255,0), -1)
    
    # multiplicação de H pelo menor ponto Y encontrado
    # matplotlib para plotar os pontos

    # imprimir frames
    cv2.imshow('Frame', concatenar_saida([
        [frame, frame_inicial, frame_fundo],
        [frame, bolinha, binarizada]
    ]))
    
    key = cv2.waitKey(fps) & 0xff
    if key == ord('q'):
        break

video.release()
cv2.destroyAllWindows() 